In [ ]:
#importing libraries
import pandas as pd
import requests
import json
import urllib.request
import time
from bs4 import BeautifulSoup
import re
import numpy as np

In [ ]:
pip install rank_bm25

In [ ]:
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
Whole_dataset=pd.read_csv(r"C:\Users\sigma\Desktop\Saner\eclipse_preproccessed_whole_dataset.csv")
Whole_dataset

,Unnamed: 0,Bug_ID,Product,Component,Status,Resolution,Summary,Changed,Description,Duplicate_Bug_Ids
0,0,518088.0,Web Tools,Web Standard Tools,RESOLVED,INVALID,search widget is not working,2017-06-10 14:42:10,search widget is not working,NaN
1,1,546444.0,z_Archived,PDT,CLOSED,NOT_ECLIPSE,Bug,2020-05-14 11:25:40,created attachment eclipse hello world,NaN
2,2,533893.0,Platform,Website,RESOLVED,INVALID,AntCompareCVSDebugDocIDEIncubatorPMCRelengReso...,2018-04-22 12:23:23,created attachment error in website antcomp...,NaN
3,3,519449.0,PDE,UI,VERIFIED,FIXED,Problem with KEY_NAME,2017-08-03 03:21:28,in product pluginsection recommendedbundles m...,NaN
4,4,519450.0,JDT,UI,RESOLVED,FIXED,Problem with KEY_NAME,2017-07-13 09:43:02,in renametyperefactoring another type the tem...,NaN
...,...,...,...,...,...,...,...,...,...,...
46259,46259,544461.0,Sirius,Server,CLOSED,WONTFIX,Add support for the update of text files,2021-01-06 05:29:02,it should be possible to update a text file us...,NaN
46260,46260,545249.0,JDT,UI,RESOLVED,FIXED,[12] AST View should not show the implicit bre...,2019-05-24 06:06:54,in the case of switchlabeledrule compiler a...,NaN
46261,46261,565646.0,Community,CI-Jenkins,CLOSED,DUPLICATE,help needed in ensuring that https://ci.eclips...,2020-08-14 09:01:13,hi are there any memory cpu usage reports fo...,565863.0
46262,46262,568788.0,z_Archived,Tycho,RESOLVED,FIXED,Support new format .tgz in tycho-p2-director:a...,2021-04-28 16:52:10,created attachment downloads for atom vscode...,NaN


In [ ]:
data = Whole_dataset.copy()

In [ ]:
data['Description1'] = data['Summary'] + data['Description']

In [ ]:
data['Description1'][3]

'Problem with KEY_NAMEin  product pluginsection recommendedbundles message the template  the following recommended start levels for plug ins will will be added  n    ncontinue   contains twice the  will  word   '

In [ ]:
data.info()

In [ ]:
data = data[["Bug_ID", "Description1", 'Duplicate_Bug_Ids', 'Resolution']]

In [ ]:
data

,Bug_ID,Description1,Duplicate_Bug_Ids,Resolution
0,518088.0,search widget is not workingsearch widget is n...,NaN,INVALID
1,546444.0,Bugcreated attachment eclipse hello world,NaN,NOT_ECLIPSE
2,533893.0,AntCompareCVSDebugDocIDEIncubatorPMCRelengReso...,NaN,INVALID
3,519449.0,Problem with KEY_NAMEin product pluginsection...,NaN,FIXED
4,519450.0,Problem with KEY_NAMEin renametyperefactoring...,NaN,FIXED
...,...,...,...,...
46259,544461.0,Add support for the update of text filesit sho...,NaN,WONTFIX
46260,545249.0,[12] AST View should not show the implicit bre...,NaN,FIXED
46261,565646.0,help needed in ensuring that https://ci.eclips...,565863.0,DUPLICATE
46262,568788.0,Support new format .tgz in tycho-p2-director:a...,NaN,FIXED


In [ ]:
data = data.rename(columns = {"Bug_ID":"Bug_Id", "Description1": "Description"})
data

,Bug_Id,Description,Duplicate_Bug_Ids,Resolution
0,518088.0,search widget is not workingsearch widget is n...,NaN,INVALID
1,546444.0,Bugcreated attachment eclipse hello world,NaN,NOT_ECLIPSE
2,533893.0,AntCompareCVSDebugDocIDEIncubatorPMCRelengReso...,NaN,INVALID
3,519449.0,Problem with KEY_NAMEin product pluginsection...,NaN,FIXED
4,519450.0,Problem with KEY_NAMEin renametyperefactoring...,NaN,FIXED
...,...,...,...,...
46259,544461.0,Add support for the update of text filesit sho...,NaN,WONTFIX
46260,545249.0,[12] AST View should not show the implicit bre...,NaN,FIXED
46261,565646.0,help needed in ensuring that https://ci.eclips...,565863.0,DUPLICATE
46262,568788.0,Support new format .tgz in tycho-p2-director:a...,NaN,FIXED


In [ ]:
data['Bug_Id']= data['Bug_Id'].values.astype('float64')

In [ ]:
data.info()

In [ ]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});') 

def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext

for index, row in data.iterrows():
    des =  str(row.Description)
    des = des.replace("\n"," ")
    des = cleanhtml(des)
    if(len(des)>=3000):
        row.Description = des[:3000]
        
#Text Cleaning round 1 (removing punctutions)
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
#     print(text)
    text = re.sub('\w*\d\w*', ' ', text)
    text = re.sub('\w*\f\w*', ' ', text)
    text = re.sub('\(.*?\)', ' ', text)
    text = re.sub('\[.*]\)', ' ', text)
    text = re.sub(r'https?://[A-Za-z0-9./]+',' ',text)
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    return text

round1 = lambda x: clean_text_round1(x)

data = data.dropna(axis=0, subset=['Description'])
data['Description'] = data['Description'].apply(clean_text_round1)

# Apply a second round of cleaning (removing punctuations)
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', ' ', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\t', ' ', text)
    return text

round2 = lambda x: clean_text_round2(x)

data['Description'] = data['Description'].apply(clean_text_round2)

In [ ]:
data

,Bug_Id,Description,Duplicate_Bug_Ids,Resolution
0,518088.0,search widget is not workingsearch widget is n...,NaN,INVALID
1,546444.0,bugcreated attachment eclipse hello world,NaN,NOT_ECLIPSE
2,533893.0,antcomparecvsdebugdocideincubatorpmcrelengreso...,NaN,INVALID
3,519449.0,problem with key namein product pluginsection...,NaN,FIXED
4,519450.0,problem with key namein renametyperefactoring...,NaN,FIXED
...,...,...,...,...
46259,544461.0,add support for the update of text filesit sho...,NaN,WONTFIX
46260,545249.0,ast view should not show the implicit brea...,NaN,FIXED
46261,565646.0,help needed in ensuring that tck doesn t use...,565863.0,DUPLICATE
46262,568788.0,support new format tgz in tycho director ar...,NaN,FIXED


In [ ]:
sentences = data['Description']

In [ ]:
tokens = []
for s in sentences:
    cleanedTex=re.sub(r'[^\w\s]','',str(s)).lower()
    words = (word_tokenize(cleanedTex))
    tokens .append(words)

In [ ]:
len(tokens)

In [ ]:
bm25 = BM25Okapi(tokens)

In [ ]:
nan_value = float("NaN")
data.replace("", nan_value, inplace=True)
data.dropna(subset = ["Description"], inplace=True)
cond = (data['Description'] == 'NaN')
cond.unique()
data = data.reset_index(drop=True)

In [ ]:
data['Description']

In [ ]:
data['Description']=data['Description'].values.astype('object')

In [ ]:
data

,Bug_Id,Description,Duplicate_Bug_Ids,Resolution
0,518088.0,search widget is not workingsearch widget is n...,NaN,INVALID
1,546444.0,bugcreated attachment eclipse hello world,NaN,NOT_ECLIPSE
2,533893.0,antcomparecvsdebugdocideincubatorpmcrelengreso...,NaN,INVALID
3,519449.0,problem with key namein product pluginsection...,NaN,FIXED
4,519450.0,problem with key namein renametyperefactoring...,NaN,FIXED
...,...,...,...,...
46258,544461.0,add support for the update of text filesit sho...,NaN,WONTFIX
46259,545249.0,ast view should not show the implicit brea...,NaN,FIXED
46260,565646.0,help needed in ensuring that tck doesn t use...,565863.0,DUPLICATE
46261,568788.0,support new format tgz in tycho director ar...,NaN,FIXED


### On complete data

In [ ]:
count=0
dup_indices_with_bug_id = {}
similarity_scores_before = []
similarity_scores = {}
issues = data.Bug_Id

for i, row in data.iterrows():
    
    issue_id = row.Bug_Id
    dup_id = row.Duplicate_Bug_Ids
    
    if dup_id not in issues.values:
        count+=1
        continue

    similarity_row = bm25.get_scores(tokens[i])
    argsort_similarity = np.argsort(similarity_row)[::-1][1:]
    similarity_scores_before = (np.sort(similarity_row)[::-1][1:])
    dup_bug_index = list(issues[issues == dup_id].index)[0]
    dup_ranking = np.argmax(argsort_similarity == np.int64(dup_bug_index))
    dup_indices_with_bug_id[issue_id] = dup_ranking

In [ ]:
count_at_1=0
count_at_5=0
count_at_10=0
count_at_20=0
count_at_25=0
count_at_30=0
count_at_50=0
count_at_75=0
count_at_100=0

for key, value in enumerate(dup_indices_with_bug_id.values()):
    if(value < 1):
        count_at_1+=1
    if(value < 5):
        count_at_5+=1
    if(value < 10):
        count_at_10+=1
    if(value < 20):
        count_at_20+=1
    if(value < 25):
        count_at_25+=1
    if(value < 30):
        count_at_30+=1
    if(value < 50):
        count_at_50+=1
    if(value < 75):
        count_at_75+=1
    if(value < 100):
        count_at_100+=1
        
print("count at 1",count_at_1)        
print("count at 5",count_at_5)
print("count at 10",count_at_10)
print("count at 20",count_at_20)
print("count at 25",count_at_25)
print("count at 30",count_at_30)
print("count at 50",count_at_50)
print("count at 75",count_at_75)
print("count at 100",count_at_100)

N = len(dup_indices_with_bug_id)

recall_rate_at_1 = count_at_1/N
print("recall_rate_at_1",recall_rate_at_1)

recall_rate_at_5 = count_at_5/N
print("recall_rate_at_5",recall_rate_at_5)

recall_rate_at_10 = count_at_10/N
print("recall_rate_at_10",recall_rate_at_10)

recall_rate_at_20 = count_at_20/N
print("recall_rate_at_20",recall_rate_at_20)

recall_rate_at_25 = count_at_25/N
print("recall_rate_at_25",recall_rate_at_25)

recall_rate_at_30 = count_at_30/N
print("recall_rate_at_30",recall_rate_at_30)

recall_rate_at_50 = count_at_50/N
print("recall_rate_at_50",recall_rate_at_50)

recall_rate_at_75 = count_at_75/N
print("recall_rate_at_75",recall_rate_at_75)

recall_rate_at_100 = count_at_100/N
print("recall_rate_at_10",recall_rate_at_100)

## Textually Similar

In [ ]:
textually_similar_data = pd.read_csv(r"C:\Users\sigma\Desktop\Saner\eclipse_final_sim.csv")
textually_similar_data

,Unnamed: 0,Bug ID,Product,Component,Status,Resolution,Summary,Average_Cos_Similarity_Trigram,Average_Cos_Similarity_Bigram,Average_Cos_Similarity_Score,Changed,Bug Id,Description,Duplicate_Bug_Ids
0,183,526539.0,Platform,Ant,CLOSED,DUPLICATE,"Executing an ""ant"" build in a non java project...",0.035231,0.043071,0.082600,2017-10-29 11:05:12,526539.0,since having a build xml file in a pl...,522581.0
1,1752,521057.0,JDT,Core,VERIFIED,DUPLICATE,Internal compiler error: java.lang.NullPointer...,0.065150,0.078524,0.102634,2018-01-02 07:11:59,521057.0,created attachment source code that causes t...,517951.0
2,1914,515976.0,z_Archived,Tycho,CLOSED,DUPLICATE,NPE during execution of org.eclipse.tycho:tych...,0.030835,0.042163,0.062864,2021-04-28 16:51:19,515976.0,we are encountering npe during this execution ...,512326.0
3,2158,516277.0,Orion,Server,CLOSED,DUPLICATE,Exception when launching server,0.030041,0.037288,0.072219,2017-06-21 11:06:14,516277.0,created attachment osgi capture since a...,517672.0
4,2177,515243.0,Platform,IDE,CLOSED,DUPLICATE,Oxygen installation via Eclipse Installer curr...,0.031605,0.038671,0.069164,2017-04-13 06:40:36,515243.0,created attachment log of eclipse installer ...,515213.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,46986,559836.0,PDE,UI,CLOSED,DUPLICATE,"A ""Search for corresponding bundle MANIFEST.MF...",0.034301,0.042108,0.076983,2020-02-04 11:05:32,559836.0,in my workspace i use nested projects and oft...,548822.0
675,47553,569402.0,Platform,Text,CLOSED,DUPLICATE,Support resolving hyperlinks without freezing ...,0.036707,0.044757,0.068871,2020-12-03 08:05:56,569402.0,see also the discussion here bug cgi id ...,506467.0
676,48115,548222.0,Equinox,Framework,RESOLVED,DUPLICATE,UnsupportedOperationException on startup of eq...,0.064757,0.074790,0.084089,2019-06-13 09:05:40,548222.0,this sometimes happen after a while the onl...,543844.0
677,48186,527632.0,JDT,Core,VERIFIED,DUPLICATE,[rawtypes] Inconsistent classfile encountered:...,0.035196,0.042290,0.077415,2018-05-01 05:42:52,527632.0,we have a large product with many plugins wi...,518157.0


In [ ]:
textually_similar_data.info()

In [ ]:
textually_similar_data['Description1'] = textually_similar_data['Summary']+ textually_similar_data['Description']

In [ ]:
textually_similar_data = textually_similar_data.rename(columns = {"Bug ID":"Bug_Id"})

In [ ]:
textually_similar_data = textually_similar_data[["Bug_Id", "Description1", 'Duplicate_Bug_Ids', 'Resolution']]

In [ ]:
textually_similar_data

In [ ]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});') 

def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext

for index, row in textually_similar_data.iterrows():
    des =  str(row.Description1)
    des = des.replace("\n"," ")
    des = cleanhtml(des)
    if(len(des)>=3000):
        row.Description1 = des[:3000]
        
#Text Cleaning round 1 (removing punctutions)
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
#     print(text)
    text = re.sub('\w*\d\w*', ' ', text)
    text = re.sub('\w*\f\w*', ' ', text)
    text = re.sub('\(.*?\)', ' ', text)
    text = re.sub('\[.*]\)', ' ', text)
    text = re.sub(r'https?://[A-Za-z0-9./]+',' ',text)
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    return text

round1 = lambda x: clean_text_round1(x)

textually_similar_data = textually_similar_data.dropna(axis=0, subset=['Description1'])
textually_similar_data['Description1'] = textually_similar_data['Description1'].apply(clean_text_round1)

# Apply a second round of cleaning (removing punctuations)
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', ' ', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\t', ' ', text)
    return text

round2 = lambda x: clean_text_round2(x)

textually_similar_data['Description1'] = textually_similar_data['Description1'].apply(clean_text_round2)

In [ ]:
textually_similar_data

In [ ]:
bm25 = BM25Okapi(tokens)

In [ ]:
nan_value = float("NaN")
textually_similar_data.replace("", nan_value, inplace=True)
textually_similar_data.dropna(subset = ["Description1"], inplace=True)
cond = (textually_similar_data['Description1'] == 'NaN')
cond.unique()
textually_similar_data = textually_similar_data.reset_index(drop=True)

In [ ]:
textually_similar_data['Description1']

In [ ]:
textually_similar_data['Description1']=textually_similar_data['Description1'].values.astype('object')

In [ ]:
textually_similar_data['Bug_Id']= textually_similar_data['Bug_Id'].values.astype('float64')

In [ ]:
sentence_sim = textually_similar_data.Description1

tokens_sim = []
for s in sentence_sim:
    cleanedTex=re.sub(r'[^\w\s]','',str(s)).lower()
    words = (word_tokenize(cleanedTex))
    tokens_sim.append(words)

In [ ]:
count=0
dup_indices_with_bug_id_similar = {}
similarity_scores_before = []
similarity_scores = {}
issues = data.Bug_Id

for i, row in textually_similar_data.iterrows():
    
    issue_id = row.Bug_Id
    dup_id = row.Duplicate_Bug_Ids
    
    if dup_id not in issues.values:
        count+=1
        continue
        
    similarity_row = bm25.get_scores(tokens_sim[i])
    argsort_similarity = np.argsort(similarity_row)[::-1][1:]
    similarity_scores_before = (np.sort(similarity_row)[::-1][1:])
    dup_bug_index = list(issues[issues == dup_id].index)[0]
    dup_ranking = np.argmax(argsort_similarity == np.int64(dup_bug_index)) 
    dup_indices_with_bug_id_similar[issue_id] = dup_ranking

In [ ]:
count_at_1=0
count_at_5=0
count_at_10=0
count_at_20=0
count_at_25=0
count_at_30=0
count_at_50=0
count_at_75=0
count_at_100=0

for key, value in enumerate(dup_indices_with_bug_id_similar.values()):
    if(value < 1):
        count_at_1+=1
    if(value < 5):
        count_at_5+=1
    if(value < 10):
        count_at_10+=1
    if(value < 20):
        count_at_20+=1
    if(value < 25):
        count_at_25+=1
    if(value < 30):
        count_at_30+=1
    if(value < 50):
        count_at_50+=1
    if(value < 75):
        count_at_75+=1
    if(value < 100):
        count_at_100+=1
        
print("count at 1",count_at_1)        
print("count at 5",count_at_5)
print("count at 10",count_at_10)
print("count at 20",count_at_20)
print("count at 25",count_at_25)
print("count at 30",count_at_30)
print("count at 50",count_at_50)
print("count at 75",count_at_75)
print("count at 100",count_at_100)

N = len(dup_indices_with_bug_id_similar)

recall_rate_at_1 = count_at_1/N
print("recall_rate_at_1",recall_rate_at_1)

recall_rate_at_5 = count_at_5/N
print("recall_rate_at_5",recall_rate_at_5)

recall_rate_at_10 = count_at_10/N
print("recall_rate_at_10",recall_rate_at_10)

recall_rate_at_20 = count_at_20/N
print("recall_rate_at_20",recall_rate_at_20)

recall_rate_at_25 = count_at_25/N
print("recall_rate_at_25",recall_rate_at_25)

recall_rate_at_30 = count_at_30/N
print("recall_rate_at_30",recall_rate_at_30)

recall_rate_at_50 = count_at_50/N
print("recall_rate_at_50",recall_rate_at_50)

recall_rate_at_75 = count_at_75/N
print("recall_rate_at_75",recall_rate_at_75)

recall_rate_at_100 = count_at_100/N
print("recall_rate_at_100",recall_rate_at_100)

# Textually Dissimilar

In [ ]:
textually_dissimilar_data = pd.read_csv(r"C:\Users\sigma\Desktop\Saner\eclipse_final_dis.csv")

In [ ]:
textually_dissimilar_data

,Unnamed: 0,Bug ID,Product,Component,Status,Resolution,Summary,Average_Cos_Similarity_Trigram,Average_Cos_Similarity_Bigram,Average_Cos_Similarity_Score,Changed,Bug Id,Description,Duplicate_Bug_Ids
0,157,576714.0,JGit,JGit,CLOSED,DUPLICATE,Upgrade org.apache.sshd:sshd-core to version 2...,0.014263,0.016080,0.027462,2021-10-19 04:47:14,576714.0,affected versions of this package are vulnerab...,574220.0
1,1372,517250.0,Community,Proposals and Reviews,CLOSED,DUPLICATE,[release] Eclipse Oxygen (4.7),0.014451,0.017776,0.028924,2017-06-12 15:52:36,517250.0,we ll use this bug to track the release we re...,517249.0
2,1721,538671.0,Community,CI-Jenkins,CLOSED,DUPLICATE,JIPP for Concurrency Utilities RI,0.013392,0.016903,0.029488,2018-09-10 12:56:59,538671.0,please create a jipp for the eclipse project...,538670.0
3,1839,527762.0,Community,Website,CLOSED,DUPLICATE,Cross site scripting,0.012121,0.013729,0.015358,2017-11-27 04:46:12,527762.0,created attachment eclise screenshot vulenr...,518274.0
4,1843,539770.0,Community,Wiki,CLOSED,DUPLICATE,"The ""allow cookies"" notice appears for every s...",0.011421,0.013845,0.023366,2019-11-21 13:52:27,539770.0,i am prompted to accept cookies with every pag...,552928.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,47922,575059.0,JDT,Core,RESOLVED,DUPLICATE,[17] flag error for non-exhaustive switch expr...,0.012449,0.015300,0.028894,2021-08-12 21:30:28,575059.0,from a switch expression must be exhaus...,575050.0
658,48043,576786.0,JDT,Core,RESOLVED,DUPLICATE,[18][switch pattern] - Conditions for Exhausti...,0.013389,0.016472,0.029899,2022-01-07 04:45:25,576786.0,section jep says t supports a sealed...,576785.0
659,48177,555561.0,Capella,UI,CLOSED,DUPLICATE,Double click shall not reveal/unreveal element,0.005806,0.006821,0.010220,2020-11-17 10:24:38,555561.0,when opening properties view double click sha...,555559.0
660,48291,526621.0,Platform,SWT,CLOSED,DUPLICATE,[GTK2] GTK: Combobox hard to read on Ubuntu 17.10,0.010740,0.013951,0.025436,2018-07-12 10:12:17,526621.0,when running smartgit on ubuntu t...,530841.0


In [ ]:
textually_dissimilar_data.info()

In [ ]:
textually_dissimilar_data['Description1'] = textually_dissimilar_data['Summary']+ textually_dissimilar_data['Description']

In [ ]:
textually_dissimilar_data = textually_dissimilar_data.rename(columns = {"Bug ID":"Bug_Id"})

In [ ]:
textually_dissimilar_data = textually_dissimilar_data[["Bug_Id", "Description1", 'Duplicate_Bug_Ids', 'Resolution']]

In [ ]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});') 

def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext

for index, row in textually_dissimilar_data.iterrows():
    des =  str(row.Description1)
    des = des.replace("\n"," ")
    des = cleanhtml(des)
    if(len(des)>=3000):
        row.Description1 = des[:3000]
        
#Text Cleaning round 1 (removing punctutions)
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
#     print(text)
    text = re.sub('\w*\d\w*', ' ', text)
    text = re.sub('\w*\f\w*', ' ', text)
    text = re.sub('\(.*?\)', ' ', text)
    text = re.sub('\[.*]\)', ' ', text)
    text = re.sub(r'https?://[A-Za-z0-9./]+',' ',text)
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    return text

round1 = lambda x: clean_text_round1(x)

textually_dissimilar_data = textually_dissimilar_data.dropna(axis=0, subset=['Description1'])
textually_dissimilar_data['Description1'] = textually_dissimilar_data['Description1'].apply(clean_text_round1)

# Apply a second round of cleaning (removing punctuations)
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', ' ', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\t', ' ', text)
    return text

round2 = lambda x: clean_text_round2(x)

textually_dissimilar_data['Description1'] = textually_dissimilar_data['Description1'].apply(clean_text_round2)

In [ ]:
textually_dissimilar_data

In [ ]:
bm25 = BM25Okapi(tokens)

In [ ]:
nan_value = float("NaN")
textually_dissimilar_data.replace("", nan_value, inplace=True)
textually_dissimilar_data.dropna(subset = ["Description1"], inplace=True)
cond = (textually_dissimilar_data['Description1'] == 'NaN')
cond.unique()
textually_dissimilar_data = textually_dissimilar_data.reset_index(drop=True)

In [ ]:
textually_dissimilar_data['Description1']=textually_dissimilar_data['Description1'].values.astype('object')

In [ ]:
textually_dissimilar_data['Bug_Id']=textually_dissimilar_data['Bug_Id'].values.astype('float64')

In [ ]:
textually_dissimilar_data

In [ ]:
sentence_dissim = textually_dissimilar_data.Description1

tokens_dissim = []
for s in sentence_dissim:
    cleanedTex=re.sub(r'[^\w\s]','',str(s)).lower()
    words = (word_tokenize(cleanedTex))
    tokens_dissim.append(words)

In [ ]:
count=0
dup_indices_with_bug_id_dissimilar = {}
similarity_scores_before = []
similarity_scores = {}

issues = data.Bug_Id 

for i, row in textually_dissimilar_data.iterrows():
    
    issue_id = row.Bug_Id
    dup_id = row.Duplicate_Bug_Ids
    
    if dup_id not in issues.values:
        count+=1
        continue
    similarity_row = bm25.get_scores(tokens_dissim[i])
    argsort_similarity = np.argsort(similarity_row)[::-1][1:]
    similarity_scores_before = (np.sort(similarity_row)[::-1][1:])
    dup_bug_index = list(issues[issues == dup_id].index)[0]
    dup_ranking = np.argmax(argsort_similarity == np.int64(dup_bug_index))
    dup_indices_with_bug_id_dissimilar[issue_id] = dup_ranking

In [ ]:
count_at_1=0
count_at_5=0
count_at_10=0
count_at_20=0
count_at_25=0
count_at_30=0
count_at_50=0
count_at_75=0
count_at_100=0

for key, value in enumerate(dup_indices_with_bug_id_dissimilar.values()):
    if(value < 1):
        count_at_1+=1
    if(value < 5):
        count_at_5+=1
    if(value < 10):
        count_at_10+=1
    if(value < 20):
        count_at_20+=1
    if(value < 25):
        count_at_25+=1
    if(value < 30):
        count_at_30+=1
    if(value < 50):
        count_at_50+=1
    if(value < 75):
        count_at_75+=1
    if(value < 100):
        count_at_100+=1
        
print("count at 1",count_at_1)        
print("count at 5",count_at_5)
print("count at 10",count_at_10)
print("count at 20",count_at_20)
print("count at 25",count_at_25)
print("count at 30",count_at_30)
print("count at 50",count_at_50)
print("count at 75",count_at_75)
print("count at 100",count_at_100)

N = len(dup_indices_with_bug_id_dissimilar)

recall_rate_at_1 = count_at_1/N
print("recall_rate_at_1",recall_rate_at_1)

recall_rate_at_5 = count_at_5/N
print("recall_rate_at_5",recall_rate_at_5)

recall_rate_at_10 = count_at_10/N
print("recall_rate_at_10",recall_rate_at_10)

recall_rate_at_20 = count_at_20/N
print("recall_rate_at_20",recall_rate_at_20)

recall_rate_at_25 = count_at_25/N
print("recall_rate_at_25",recall_rate_at_25)

recall_rate_at_30 = count_at_30/N
print("recall_rate_at_30",recall_rate_at_30)

recall_rate_at_50 = count_at_50/N
print("recall_rate_at_50",recall_rate_at_50)

recall_rate_at_75 = count_at_75/N
print("recall_rate_at_75",recall_rate_at_75)

recall_rate_at_100 = count_at_100/N
print("recall_rate_at_100",recall_rate_at_100)